### Job ETL: Raw -> Silver

Notebook responsavel por extrair dados da camada raw, higienizar e entregar na camada silver. Execute as celulas na ordem apresentada; ajuste apenas os parametros identificados.


#### Importação das bibliotecas

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, when, split
from pyspark.sql.types import DoubleType, IntegerType
import os

#### Inicialização da SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Formula1_Raw_to_Silver_Outliers") \
    .getOrCreate()

#### Definição dos caminhos

Definimos os caminhos para as camadas Raw (origem) e Silver (destino) e garantimos que o diretório de destino exista.

In [4]:
base_path = os.path.join("C:", os.sep, "Users", "julii", "OneDrive", "Documentos", "formula1-analytics", "Data Layer")
raw_path = os.path.join(base_path, "raw", "dados_originais")
silver_path = os.path.join(base_path, "silver", "dados_limpos")

os.makedirs(silver_path, exist_ok=True)
print(f"Diretório de origem (RAW): {raw_path}")
print(f"Diretório de destino (SILVER): {silver_path}\n")

Diretório de origem (RAW): C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\raw\dados_originais
Diretório de destino (SILVER): C:\Users\julii\OneDrive\Documentos\formula1-analytics\Data Layer\silver\dados_limpos



#### Leitura dos arquivos da camada Raw

Todos os arquivos CSV da camada Raw foram carregados em DataFrames do Apache Spark. A utilização do parâmetro nullValue="\\N" é essencial para garantir o tratamento adequado dos valores nulos presentes no conjunto de dados.

In [5]:
print(" Lendo arquivos da camada RAW com tratamento de nulos (\\N)... \n")

df_lap_times = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "lap_times.csv"), header=True, inferSchema=True)
df_results = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "results.csv"), header=True, inferSchema=True)
df_races = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "races.csv"), header=True, inferSchema=True)
df_drivers = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "drivers.csv"), header=True, inferSchema=True)
df_constructors = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "constructors.csv"), header=True, inferSchema=True)
df_status = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "status.csv"), header=True, inferSchema=True)
df_pit_stops = spark.read.option("nullValue", "\\N").csv(os.path.join(raw_path, "pit_stops.csv"), header=True, inferSchema=True)

print(" Arquivos RAW carregados com sucesso!\n")

 Lendo arquivos da camada RAW com tratamento de nulos (\N)... 

 Arquivos RAW carregados com sucesso!



#### Limpeza, Padronização e Remoção de Outliers

As colunas das tabelas races, drivers e constructors foram limpas e renomeadas para padronização e melhor legibilidade dos dados.

In [6]:
print(" Limpando, padronizando e removendo outliers para a camada Silver...\n")

races_silver = df_races.select(
    col("raceId").alias("id_corrida"),
    col("year").alias("ano"),
    col("round").alias("rodada"),
    col("name").alias("nome_corrida")
)

drivers_silver = df_drivers.select(
    col("driverId").alias("id_piloto"),
    col("forename").alias("primeiro_nome_piloto"),
    col("surname").alias("sobrenome_piloto")
)

constructors_silver = df_constructors.select(
    col("constructorId").alias("id_equipe"),
    col("name").alias("nome_equipe")
)

 Limpando, padronizando e removendo outliers para a camada Silver...



#### Tratamento e Remoção de Outliers em pit_stops

Foi aplicada uma lógica robusta para converter a coluna duration — que apresenta variações de formato — em valores numéricos correspondentes a segundos. Em seguida, realizou-se a remoção de outliers utilizando o método IQR (Interquartile Range), garantindo maior consistência e qualidade aos dados.

In [7]:
pit_stops_cleaned = df_pit_stops.withColumn(
    "duracao_parada_seg",
    when(
        col("duration").contains(":"),
        split(col("duration"), ":").getItem(0).cast("double") * 60 +
        split(col("duration"), ":").getItem(1).cast("double")
    ).otherwise(
        col("duration").cast("double")
    )
)

pit_stops_silver = pit_stops_cleaned.select(
    col("raceId").alias("id_corrida"),
    col("driverId").alias("id_piloto"),
    "duracao_parada_seg"
).na.drop(subset=["duracao_parada_seg"])

print("    → Removendo outliers de 'pit_stops' (duração)...")
if pit_stops_silver.count() > 0:
    quantiles_pit = pit_stops_silver.stat.approxQuantile("duracao_parada_seg", [0.25, 0.75], 0.01)
    q1_pit, q3_pit = quantiles_pit[0], quantiles_pit[1]
    if q1_pit is not None and q3_pit is not None:
        iqr_pit = q3_pit - q1_pit
        lower_bound_pit = q1_pit - (1.5 * iqr_pit)
        upper_bound_pit = q3_pit + (1.5 * iqr_pit)
        count_before_pit = pit_stops_silver.count()
        pit_stops_silver = pit_stops_silver.filter(
            (col("duracao_parada_seg") >= lower_bound_pit) & (col("duracao_parada_seg") <= upper_bound_pit)
        )
        count_after_pit = pit_stops_silver.count()
        print(f"      {count_before_pit - count_after_pit} outliers removidos.")
    else:
        print("      Não foi possível calcular os quantis para 'pit_stops'; pulando a remoção de outliers.")
else:
    print("      DataFrame 'pit_stops' vazio após a limpeza; pulando a remoção de outliers.")

    → Removendo outliers de 'pit_stops' (duração)...
      1036 outliers removidos.


#### Limpeza de status e results

As colunas das tabelas status e results foram limpas e renomeadas para assegurar padronização e consistência nos dados.

In [8]:
status_silver = df_status.select(
    col("statusId").alias("id_status"),
    col("status").alias("descricao_status")
)

results_silver = df_results.select(
    col("raceId").alias("id_corrida"),
    col("driverId").alias("id_piloto"),
    col("constructorId").alias("id_equipe"),
    col("statusId").alias("id_status")
)

#### Tratamento e Remoção de Outliers na Tabela Fato lap_times

A tabela fato (lap_times_fact) foi limpa, assegurando que os valores de milissegundos fossem convertidos para formato numérico. Além disso, realizou-se a remoção de outliers relacionados ao tempo de volta, visando aprimorar a qualidade e a precisão dos dados.

In [9]:
lap_times_fact = df_lap_times.select(
    col("raceId").alias("id_corrida"),
    col("driverId").alias("id_piloto"),
    col("lap").alias("volta"),
    col("position").alias("posicao_na_volta"),
    expr("try_cast(milliseconds as integer)").alias("tempo_volta_ms")
).na.drop(subset=["tempo_volta_ms"])

print("    → Removendo outliers de 'lap_times' (tempo da volta)...")
if lap_times_fact.count() > 0:
    quantiles_lap = lap_times_fact.stat.approxQuantile("tempo_volta_ms", [0.25, 0.75], 0.01)
    q1_lap, q3_pit = quantiles_lap[0], quantiles_lap[1]
    if q1_lap is not None and q3_pit is not None:
        iqr_lap = q3_pit - q1_lap
        lower_bound_lap = q1_lap - (1.5 * iqr_lap)
        upper_bound_lap = q3_pit + (1.5 * iqr_lap)
        count_before_lap = lap_times_fact.count()
        lap_times_fact = lap_times_fact.filter(
            (col("tempo_volta_ms") >= lower_bound_lap) & (col("tempo_volta_ms") <= upper_bound_lap)
        ).orderBy(col("id_corrida"), col("id_piloto"), col("volta"))
        count_after_pit = lap_times_fact.count()
        print(f"      {count_before_lap - count_after_pit} outliers removidos.")
    else:
        print("      Não foi possível calcular os quantis para 'lap_times'; pulando a remoção de outliers.")
else:
    print("      DataFrame 'lap_times' vazio após a limpeza; pulando a remoção de outliers.")

print("\n Todas as tabelas Silver foram processadas!\n")

    → Removendo outliers de 'lap_times' (tempo da volta)...
      18174 outliers removidos.

 Todas as tabelas Silver foram processadas!



#### Carga (Salvar camada Silver)

Nesta etapa, cada DataFrame processado foi salvo como um arquivo CSV individual na pasta correspondente à camada Silver, garantindo a organização e a separação lógica dos dados tratados.

In [10]:
print(" Salvando todas as tabelas na camada SILVER...\n")

silver_tables = {
    "races": races_silver,
    "drivers": drivers_silver,
    "constructors": constructors_silver,
    "pit_stops": pit_stops_silver,
    "status": status_silver,
    "results": results_silver,
    "lap_times_fact": lap_times_fact
}

for name, df in silver_tables.items():
    file_path = os.path.join(silver_path, f"{name}.csv")
    df.coalesce(1).toPandas().to_csv(file_path, index=False, header=True)
    print(f"  → Arquivo salvo: {name}.csv")

print("\n Camada Silver gerada com sucesso!\n")

 Salvando todas as tabelas na camada SILVER...

  → Arquivo salvo: races.csv
  → Arquivo salvo: drivers.csv
  → Arquivo salvo: constructors.csv
  → Arquivo salvo: pit_stops.csv
  → Arquivo salvo: status.csv
  → Arquivo salvo: results.csv
  → Arquivo salvo: lap_times_fact.csv

 Camada Silver gerada com sucesso!



#### Validação

Por fim, o resultado foi validado por meio da exibição do schema e das primeiras linhas da tabela fato principal, assegurando que todo o processo de transformação e limpeza dos dados foi executado com sucesso.

In [11]:
print(" Validando a tabela fato principal 'lap_times_fact':\n")
lap_times_fact.printSchema()
lap_times_fact.show(10)

print("\n Job ETL (Raw → Silver) finalizado com sucesso!")

 Validando a tabela fato principal 'lap_times_fact':

root
 |-- id_corrida: integer (nullable = true)
 |-- id_piloto: integer (nullable = true)
 |-- volta: integer (nullable = true)
 |-- posicao_na_volta: integer (nullable = true)
 |-- tempo_volta_ms: integer (nullable = true)

+----------+---------+-----+----------------+--------------+
|id_corrida|id_piloto|volta|posicao_na_volta|tempo_volta_ms|
+----------+---------+-----+----------------+--------------+
|         1|        1|    1|              13|        109088|
|         1|        1|    2|              12|         93740|
|         1|        1|    3|              11|         91600|
|         1|        1|    4|              10|         91067|
|         1|        1|    5|              10|         92129|
|         1|        1|    6|               9|         90469|
|         1|        1|    7|               9|         89488|
|         1|        1|    8|               9|         90302|
|         1|        1|    9|               9|     